In [1]:
import os
import pandas as pd

In [170]:
df_gseis = pd.read_csv('../data/serenelli2017/s17_seis.csv')
df_scarel = pd.read_csv('../data/asteroseismology/s17_scaling_relations_raw.csv')
df_seis = df_gseis.merge(df_scarel, on='KIC')

scale = 'ASPC'
df_spec = pd.read_csv(f'../data/spectroscopy/s17_gaia_4arcsec_apogeeDR14.csv')
df_spec = df_spec.merge(pd.read_csv(f'../data/serenelli2017/s17_{scale}.csv'), on='KIC')

df_gaia = pd.read_csv('../data/kepler_gaia/s17_gaia_4arcsec.csv')

# Merge on kepler ID and nearest neighbour to account for duplicates
df_comb = df_gaia.merge(df_spec, on=['KIC', 'nearest_neighbour'], suffixes=['_GAIA', f'_{scale}'])
# df_comb = df_gaia.merge(df_spec, on='KIC', suffixes=['_GAIA', f'_{scale}'])

cond = df_comb['nearest_neighbour'] == 1  # Choose only nearest neighbours
df = df_seis.merge(df_comb[cond], on='KIC')
df.head()

,KIC,numax,enumax,Dnu,eDnu,Length,HBR,logg_seis_SDSS,logg_seis_err_SDSS,logg_seis_ASPC,...,-elogg,sylogg,rho,+erho,-erho,syrho,Age,+eAge,-eAge,syAge
0,1435467,1382.311,19.038,70.558,0.087,938.1,1.662,4.111703,0.007873,4.100044,...,-0.005,0.004,0.27110,0.001138,-0.001015,0.001526,3.746,0.349,-0.324,0.496
1,2010607,674.919,146.523,42.479,2.185,28.9,1.547,3.797905,0.094409,3.785706,...,-0.027,0.006,0.09667,0.009696,-0.009474,0.001512,3.544,0.396,-0.390,0.333
2,2309595,643.208,11.226,39.029,0.721,28.9,1.767,3.734825,0.009083,3.724735,...,-0.008,0.007,0.08390,0.002592,-0.002614,0.001254,9.537,2.667,-2.196,2.517
3,2450729,1053.105,114.904,61.910,2.539,28.9,1.470,3.979485,0.047620,3.972583,...,-0.021,0.007,0.20350,0.015640,-0.015730,0.002757,7.724,1.051,-0.953,1.221
4,2837475,1629.761,18.396,75.720,0.129,1029.4,1.767,4.191662,0.006726,4.179901,...,-0.007,0.020,0.32450,0.002867,-0.002731,0.003638,1.804,0.313,-0.285,0.529


In [171]:
df.columns.to_list()

['KIC',
 'numax',
 'enumax',
 'Dnu',
 'eDnu',
 'Length',
 'HBR',
 'logg_seis_SDSS',
 'logg_seis_err_SDSS',
 'logg_seis_ASPC',
 'logg_seis_err_ASPC',
 'logg_seis_DR14',
 'logg_seis_err_DR14',
 'logg_seis_DR16',
 'logg_seis_err_DR16',
 'R_seis_SDSS',
 'R_seis_err_SDSS',
 'R_seis_ASPC',
 'R_seis_err_ASPC',
 'R_seis_DR14',
 'R_seis_err_DR14',
 'R_seis_DR16',
 'R_seis_err_DR16',
 'M_seis_SDSS',
 'M_seis_err_SDSS',
 'M_seis_ASPC',
 'M_seis_err_ASPC',
 'M_seis_DR14',
 'M_seis_err_DR14',
 'M_seis_DR16',
 'M_seis_err_DR16',
 'solution_id',
 'designation',
 'source_id',
 'random_index',
 'gaia_ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 

We need to specify the columns for input.

In [172]:
df['parallax']

0      7.287584
1      1.578592
2      2.547900
3      3.636056
4      8.247791
         ...   
408    1.769590
409    4.817379
410    3.213275
411    6.689642
412    2.898009
Name: parallax, Length: 413, dtype: float64

In [173]:
parallax_offset = 0.05  # mas
df['parallax_cor'] = df['parallax'] + parallax_offset

In [174]:
df['parallax_cor_as'] = df['parallax_cor'] * 1e-3
df['parallax_error_as'] = df['parallax_error'] * 1e-3

In [175]:
df['band'] = 'kmag'

In [176]:
df['dust'] = 'green19'

In [177]:
df['comments'] = f'using DR13 feh and {scale} teff'

In [178]:
df['id_starname'] = 'KIC' + df['KIC'].astype(str)

Dict of columns in master table with their respective columns in the inputs.

In [179]:
cols = {
    'id_starname': 'id_starname', 
    'Teff': 'teff',
    'eTeff': 'teff_err',
    f'logg_seis_{scale}': 'logg',
    f'logg_seis_err_{scale}': 'logg_err',
    '[M/H]': 'feh',
    'e[M/H]': 'feh_err',
    'parallax_cor_as': 'parallax',
    'parallax_error_as': 'parallax_err',
    'J': 'jmag',
    'J_ERR': 'jmag_err',
    'K': 'kmag',
    'K_ERR': 'kmag_err',
    'ra': 'ra',
    'dec': 'dec',
    'band': 'band',
    'dust': 'dust',
    'comments': 'comments',
#     'dmag': 'dmag',
#     'dmag_err': 'dmag_err'
}

In [180]:
inputs = df[cols.keys()]
inputs.head()

,id_starname,Teff,eTeff,logg_seis_ASPC,logg_seis_err_ASPC,[M/H],e[M/H],parallax_cor_as,parallax_error_as,J,J_ERR,K,K_ERR,ra,dec,band,dust,comments
0,KIC1435467,6096.7,69.276,4.100044,0.007723,-0.028,0.099,0.007338,0.000030,7.983,0.024,7.718,0.017,292.082639,37.059874,kmag,green19,using DR13 feh and ASPC teff
1,KIC2010607,6013.5,69.276,3.785706,0.094411,-0.070,0.100,0.001629,0.000026,10.427,0.020,10.155,0.019,290.505591,37.458935,kmag,green19,using DR13 feh and ASPC teff
2,KIC2309595,5000.2,69.276,3.724735,0.009181,-0.095,0.101,0.002598,0.000028,10.040,0.022,9.500,0.018,292.901449,37.624510,kmag,green19,using DR13 feh and ASPC teff
3,KIC2450729,5840.4,69.276,3.972583,0.047643,-0.212,0.100,0.003686,0.000029,9.585,0.020,9.283,0.018,293.073631,37.716582,kmag,green19,using DR13 feh and ASPC teff
4,KIC2837475,6335.4,69.276,4.179901,0.006888,-0.015,0.100,0.008298,0.000029,7.649,0.021,7.464,0.027,287.548574,38.082134,kmag,green19,using DR13 feh and ASPC teff


In [181]:
iso_inputs = inputs.rename(columns=cols)
iso_inputs.head()

,id_starname,teff,teff_err,logg,logg_err,feh,feh_err,parallax,parallax_err,jmag,jmag_err,kmag,kmag_err,ra,dec,band,dust,comments
0,KIC1435467,6096.7,69.276,4.100044,0.007723,-0.028,0.099,0.007338,0.000030,7.983,0.024,7.718,0.017,292.082639,37.059874,kmag,green19,using DR13 feh and ASPC teff
1,KIC2010607,6013.5,69.276,3.785706,0.094411,-0.070,0.100,0.001629,0.000026,10.427,0.020,10.155,0.019,290.505591,37.458935,kmag,green19,using DR13 feh and ASPC teff
2,KIC2309595,5000.2,69.276,3.724735,0.009181,-0.095,0.101,0.002598,0.000028,10.040,0.022,9.500,0.018,292.901449,37.624510,kmag,green19,using DR13 feh and ASPC teff
3,KIC2450729,5840.4,69.276,3.972583,0.047643,-0.212,0.100,0.003686,0.000029,9.585,0.020,9.283,0.018,293.073631,37.716582,kmag,green19,using DR13 feh and ASPC teff
4,KIC2837475,6335.4,69.276,4.179901,0.006888,-0.015,0.100,0.008298,0.000029,7.649,0.021,7.464,0.027,287.548574,38.082134,kmag,green19,using DR13 feh and ASPC teff


In [183]:
import numpy as np

iso_inputs.replace([-9999.0, -9999.99, -999.99], np.nan, inplace=True)
iso_inputs_dropna = iso_inputs.dropna()

In [184]:
len(iso_inputs_dropna)

408

Drop negative parallax

In [185]:
iso_inputs_dropna[iso_inputs_dropna['parallax']<0]

,id_starname,teff,teff_err,logg,logg_err,feh,feh_err,parallax,parallax_err,jmag,jmag_err,kmag,kmag_err,ra,dec,band,dust,comments
98,KIC5689219,6294.8,69.276,4.080307,0.039822,-0.307,0.1,-0.000537,0.000691,8.983,0.026,8.697,0.015,285.945577,40.920561,kmag,green19,using DR13 feh and ASPC teff


In [186]:
iso_inputs_dropnegpar = iso_inputs_dropna[iso_inputs_dropna['parallax']>0]

In [187]:
len(iso_inputs_dropnegpar)

407

In [188]:
iso_inputs_dropnegpar.to_csv(f'../data/isoclassify/isoclassify_inputs_DR13_{scale}.csv', index=False)

In [195]:
df_iso = pd.read_csv('../data/isoclassify/isoclassify_inputs_DR13_SDSS.csv')
df_iso

,id_starname,teff,teff_err,logg,logg_err,feh,feh_err,parallax,parallax_err,jmag,jmag_err,kmag,kmag_err,ra,dec,band,dust,comments
0,KIC1435467,6433.0,86.0,4.111703,0.007873,-0.028,0.099,0.007338,0.000030,7.983,0.024,7.718,0.017,292.082639,37.059874,kmag,green19,using DR13 feh and SDSS teff
1,KIC2010607,6361.0,71.0,3.797905,0.094409,-0.070,0.100,0.001629,0.000026,10.427,0.020,10.155,0.019,290.505591,37.458935,kmag,green19,using DR13 feh and SDSS teff
2,KIC2309595,5238.0,65.0,3.734825,0.009083,-0.095,0.101,0.002598,0.000028,10.040,0.022,9.500,0.018,292.901449,37.624510,kmag,green19,using DR13 feh and SDSS teff
3,KIC2450729,6029.0,59.0,3.979485,0.047620,-0.212,0.100,0.003686,0.000029,9.585,0.020,9.283,0.018,293.073631,37.716582,kmag,green19,using DR13 feh and SDSS teff
4,KIC2837475,6688.0,57.0,4.191662,0.006726,-0.015,0.100,0.008298,0.000029,7.649,0.021,7.464,0.027,287.548574,38.082134,kmag,green19,using DR13 feh and SDSS teff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,KIC12069569,6357.0,59.0,3.836015,0.060734,-0.109,0.100,0.001820,0.000025,10.286,0.021,10.039,0.014,295.529027,50.527485,kmag,green19,using DR13 feh and SDSS teff
403,KIC12117868,6160.0,51.0,4.231000,0.043871,-0.108,0.099,0.004867,0.000022,9.486,0.021,9.218,0.015,295.251250,50.694732,kmag,green19,using DR13 feh and SDSS teff
404,KIC12265063,6189.0,60.0,4.084861,0.076210,-0.268,0.101,0.003263,0.000028,9.939,0.021,9.626,0.016,295.070548,50.903640,kmag,green19,using DR13 feh and SDSS teff
405,KIC12317678,6549.0,48.0,4.071399,0.006796,-0.300,0.204,0.006740,0.000088,7.883,0.037,7.631,0.017,296.657238,51.020181,kmag,green19,using DR13 feh and SDSS teff
